In [53]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight

In [54]:
gc.enable()

In [55]:
steps = 20
height = 108
width = 108

In [56]:
#change the path to where you save the input numpy arrays
path = 'E:/notes_database/75_overlap/'
lstm = np.array(glob.glob(path+'lstm_inputs/*'))
cnn = np.array(glob.glob(path+'cnn_inputs/*'))
y_train = np.array(glob.glob(path+'y_train/*'))
c_lstm = np.array(glob.glob(path+'counter_lstm_inputs/*'))
c_cnn = np.array(glob.glob(path+'counter_cnn_inputs/*'))
c_y_train = np.array(glob.glob(path+'counter_y_train/*'))

In [57]:
#The function analyzes the actual accuracy
def load_test_score(model, cnn_test, lstm_test, y_test):
    prediction = model.predict([cnn_test, lstm_test])
    error = (prediction - y_test) ** 2
    error = (error/y_test.shape[0]).sum()
    wrong = 0
    for x in range(prediction.shape[0]):
        for y in range(prediction.shape[1]):
            if abs(prediction[x][y] - y_test[x][y]) > 0.5:
                wrong += 1
                break
    accuracy = 1 - wrong/(prediction.shape[0])
    return error, accuracy

In [58]:
def inputs_to_model(order, num, start, ending):
    cnn_inputs = np.load(cnn[order[num][0]])
    labels_inputs = np.load(y_train[order[num][0]])
    lstm_inputs = np.load(lstm[order[num][0]])
    
    lstm_inputs = np.append(lstm_inputs, np.load(c_lstm[order[num][0]]), axis=0)
    cnn_inputs = np.append(cnn_inputs, np.load(c_cnn[order[num][0]]), axis=0)
    labels_inputs = np.append(labels_inputs, np.load(c_y_train[order[num][0]]), axis=0)
    
    for file in order[num][start+1:ending]:
        
        lstm_inputs = np.append(lstm_inputs, np.load(lstm[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(y_train[file]), axis=0)
        lstm_inputs = np.append(lstm_inputs, np.load(c_lstm[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(c_cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(c_y_train[file]), axis=0)
    
    lstm_inputs = lstm_inputs.astype(np.float16)
    labels_inputs = labels_inputs.astype(np.float16)
    return lstm_inputs, cnn_inputs, labels_inputs

In [59]:
def main(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [60]:
def main_2(optimizer, loss):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = Dropout(0.3)(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = Flatten()(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

In [61]:
def main_3(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [62]:
def train_test_whole(classifier, num, name, epochs=100, batchsize=32):
    lstm_inputs, cnn_inputs, labels_inputs = inputs_to_model(train_order, num, 0, train_order[num].shape[0]+1)
    
    classifier.fit([cnn_inputs, lstm_inputs], labels_inputs, epochs=epochs, batch_size=batchsize, verbose=1,
                  validation_data = ([cnn_test, lstm_test], labels_test))
    
    #CV_loss, accuracy = load_test_score(classifier, cnn_test, lstm_test, y_test)
    #print('val_loss: ', CV_loss)
    #print('val_accuarcy', accuracy)
    
    classifier.save_weights(name, overwrite=True)   
    del classifier
    K.clear_session()

In [63]:
#Generate the CV sets
kf = KFold(n_splits=3, random_state=7)

train_order = []
test_order = []
length = np.arange(len(lstm))

for x,y in kf.split(length):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [64]:
lstm_test, cnn_test, labels_test = inputs_to_model(test_order, 0, 0, test_order[0].shape[0]+1)

50


In [65]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = main(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-75-overlap-model1.hdf5', epochs=50)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


50
100
Train on 145313 samples, validate on 73497 samples
Epoch 1/50
145313/145313 [==============================] - 264s 2ms/step - loss: 0.0574 - acc: 0.9792 - val_loss: 0.0205 - val_acc: 0.9936
Epoch 2/50
145313/145313 [==============================] - 259s 2ms/step - loss: 0.0208 - acc: 0.9933 - val_loss: 0.0143 - val_acc: 0.9955
Epoch 3/50
145313/145313 [==============================] - 254s 2ms/step - loss: 0.0161 - acc: 0.9950 - val_loss: 0.0185 - val_acc: 0.9947
Epoch 4/50
145313/145313 [==============================] - 259s 2ms/step - loss: 0.0143 - acc: 0.9956 - val_loss: 0.0344 - val_acc: 0.9882
Epoch 5/50
145313/145313 [==============================] - 259s 2ms/step - loss: 0.0124 - acc: 0.9962 - val_loss: 0.0113 - val_acc: 0.9966
Epoch 6/50
145313/145313 [==============================] - 258s 2ms/step - loss: 0.0118 - acc: 0.9965 - val_loss: 0.0076 - val_acc: 0.9978
Epoch 7/50
145313/145313 [==============================] - 255s 2ms/step - loss: 0.0109 - acc: 0.9967

39738

In [66]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = main_3(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-75-overlap-model2.hdf5', epochs=50)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


50
100
Train on 145313 samples, validate on 73497 samples
Epoch 1/50
145313/145313 [==============================] - 260s 2ms/step - loss: 0.0292 - acc: 0.9901 - val_loss: 0.0137 - val_acc: 0.9957
Epoch 2/50
145313/145313 [==============================] - 258s 2ms/step - loss: 0.0141 - acc: 0.9956 - val_loss: 0.0108 - val_acc: 0.9967
Epoch 3/50
145313/145313 [==============================] - 257s 2ms/step - loss: 0.0113 - acc: 0.9964 - val_loss: 0.0092 - val_acc: 0.9972
Epoch 4/50
145313/145313 [==============================] - 255s 2ms/step - loss: 0.0097 - acc: 0.9969 - val_loss: 0.0092 - val_acc: 0.9972
Epoch 5/50
145313/145313 [==============================] - 256s 2ms/step - loss: 0.0086 - acc: 0.9973 - val_loss: 0.0101 - val_acc: 0.9971
Epoch 6/50
145313/145313 [==============================] - 255s 2ms/step - loss: 0.0088 - acc: 0.9973 - val_loss: 0.0106 - val_acc: 0.9972
Epoch 7/50
145313/145313 [==============================] - 252s 2ms/step - loss: 0.0072 - acc: 0.9977

26486